# EEG Source Reconstruction Pipeline

This script loads EEG epochs from an EEGLAB `.set` file, sets up a standard 64-channel montage, and computes the forward solution for source localization using the `fsaverage` template.


In [1]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import pandas as pd

## 1. Load EEG Epochs

We load preprocessed EEG data stored in an EEGLAB `.set` file and convert it to an MNE `Epochs` object for further processing.


In [2]:
raw_data_path = '../data/raw/'
path = raw_data_path + 'PPT1/'
input_fname = path + 's_101_Coordination.set'

In [3]:
epochs = mne.io.read_epochs_eeglab(input_fname)
print(epochs)

epochs = epochs[:5]

Extracting parameters from /Users/s204684/Work/Special Course 2/Mirror_LEiDA/notebooks/../data/raw/PPT1/s_101_Coordination.set...
Not setting metadata
87 matching events found
No baseline correction applied
0 projection items activated


/var/folders/v3/57xh39sd4g107fc_xrn2v6xc0000gp/T/ipykernel_28986/3537510787.py:1: RuntimeWarning: At least one epoch has multiple events. Only the latency of the first event will be retained.
  epochs = mne.io.read_epochs_eeglab(input_fname)
/var/folders/v3/57xh39sd4g107fc_xrn2v6xc0000gp/T/ipykernel_28986/3537510787.py:1: RuntimeWarning: Data file name in EEG.data (s_01_Coordination_AverageReref.fdt) is incorrect, the file name must have changed on disk, using the correct file name (s_101_Coordination.fdt).
  epochs = mne.io.read_epochs_eeglab(input_fname)


Ready.
<EpochsEEGLAB | 87 events (all good), -1 – 4.996 s (baseline off), ~65.3 MiB, data loaded,
 '154/134/134': 32
 '134/114/154/134': 16
 '114/154/134/134': 24
 '134/154/134': 15>


## 2. Set Montage

We apply the standard 64-channel BioSemi montage to ensure correct electrode positioning in 3D space. This step is crucial for accurate source localization.


In [4]:
# set montage
montage = mne.channels.make_standard_montage('biosemi64')
epochs.set_montage(montage)

# IMPORTANT: Set a standard reference recognized by MNE




<EpochsEEGLAB | 5 events (all good), -1 – 4.996 s (baseline off), ~3.8 MiB, data loaded,
 '154/134/134': 2
 '134/114/154/134': 1
 '114/154/134/134': 1
 '134/154/134': 1>

## 3. Compute Forward Solution

The forward solution maps neural sources in the brain to EEG scalp signals. We use the `fsaverage` template, which includes:
- A predefined **source space** (dipole grid on the cortex).
- A **BEM model** (boundary element model of the head).
- A standard **head-to-MRI transform** (`fsaverage`).

The forward model is computed with `mne.make_forward_solution()`, ensuring sources are at least **5 mm** away from the inner skull.


In [5]:
subjects_dir = mne.datasets.fetch_fsaverage()
print(f"Fsaverage directory is at: {subjects_dir}")

trans = 'fsaverage'  # MNE has a built-in fsaverage transformation
src = os.path.join(subjects_dir, 'bem', 'fsaverage-ico-5-src.fif')
bem = os.path.join(subjects_dir, 'bem', 'fsaverage-5120-5120-5120-bem-sol.fif')

# Build the forward solution
fwd = mne.make_forward_solution(
    info=epochs.info,
    trans=trans,
    src=src,
    bem=bem,
    eeg=True,
    mindist=5.0,
    n_jobs=4
)

print(fwd)


0 files missing from root.txt in /Users/s204684/mne_data/MNE-fsaverage-data
0 files missing from bem.txt in /Users/s204684/mne_data/MNE-fsaverage-data/fsaverage
Fsaverage directory is at: /Users/s204684/mne_data/MNE-fsaverage-data/fsaverage
Source space          : /Users/s204684/mne_data/MNE-fsaverage-data/fsaverage/bem/fsaverage-ico-5-src.fif
MRI -> head transform : /Users/s204684/miniconda3/envs/mirror/lib/python3.12/site-packages/mne/data/fsaverage/fsaverage-trans.fif
Measurement data      : instance of Info
Conductor model   : /Users/s204684/mne_data/MNE-fsaverage-data/fsaverage/bem/fsaverage-5120-5120-5120-bem-sol.fif
Accurate field computations
Do computations in head coordinates
Free source orientations

Reading /Users/s204684/mne_data/MNE-fsaverage-data/fsaverage/bem/fsaverage-ico-5-src.fif...
Read 2 source spaces a total of 20484 active source locations

Coordinate transformation: MRI (surface RAS) -> head
    0.999994 0.003552 0.000202      -1.76 mm
    -0.003558 0.998389 0.0

/var/folders/v3/57xh39sd4g107fc_xrn2v6xc0000gp/T/ipykernel_28986/945290720.py:9: RuntimeWarning: joblib not installed. Cannot run in parallel.
  fwd = mne.make_forward_solution(


    Found     0/ 7809 points outside using solid angles
    Total 10242/10242 points inside the surface
Interior check completed in 7436.3 ms
Checking surface interior status for 10242 points...
    Found  2241/10242 points inside  an interior sphere of radius   47.7 mm
    Found     0/10242 points outside an exterior sphere of radius   98.3 mm
    Found     0/ 8001 points outside using surface Qhull


/var/folders/v3/57xh39sd4g107fc_xrn2v6xc0000gp/T/ipykernel_28986/945290720.py:9: RuntimeWarning: joblib not installed. Cannot run in parallel.
  fwd = mne.make_forward_solution(


    Found     0/ 8001 points outside using solid angles
    Total 10242/10242 points inside the surface
Interior check completed in 7565.2 ms

Setting up for EEG...
Computing EEG at 20484 source locations (free orientations)...


/var/folders/v3/57xh39sd4g107fc_xrn2v6xc0000gp/T/ipykernel_28986/945290720.py:9: RuntimeWarning: joblib not installed. Cannot run in parallel.
  fwd = mne.make_forward_solution(



Finished.
<Forward | MEG channels: 0 | EEG channels: 64 | Source space: Surface with {self['nsource']} vertices | Source orientation: Free>


## 4. Compute Noise Covariance & Inverse Operator

In [6]:
noise_cov = mne.make_ad_hoc_cov(epochs.info)

inverse_operator = mne.minimum_norm.make_inverse_operator(
    info=epochs.info,
    forward=fwd,
    noise_cov=noise_cov,
    loose=1.0,
    depth=0.8,
    verbose=True
)

Computing inverse operator with 64 channels.
    64 out of 64 channels remain after picking
Selected 64 channels
Creating the depth weighting matrix...
    64 EEG channels
    limit = 20485/20484 = 2.329337
    scale = 108975 exp = 0.8
Whitening the forward solution.
Computing rank from covariance with rank=None
    Using tolerance 5.7e-18 (2.2e-16 eps * 64 dim * 0.0004  max singular value)
    Estimated rank (eeg): 64
    EEG: rank 64 computed from 64 data channels with 0 projectors
    Setting small EEG eigenvalues to zero (without PCA)
Creating the source covariance matrix
Adjusting source covariance matrix.
Computing SVD of whitened and weighted lead field matrix.


/var/folders/v3/57xh39sd4g107fc_xrn2v6xc0000gp/T/ipykernel_28986/1267579594.py:3: RuntimeWarning: No average EEG reference present in info["projs"], covariance may be adversely affected. Consider recomputing covariance using with an average eeg reference projector added.
  inverse_operator = mne.minimum_norm.make_inverse_operator(
/var/folders/v3/57xh39sd4g107fc_xrn2v6xc0000gp/T/ipykernel_28986/1267579594.py:3: RuntimeWarning: No average EEG reference present in info["projs"], covariance may be adversely affected. Consider recomputing covariance using with an average eeg reference projector added.
  inverse_operator = mne.minimum_norm.make_inverse_operator(


    largest singular value = 4.72502
    scaling factor to adjust the trace = 1.44743e+23 (nchan = 64 nzero = 0)


## 5. Apply the Inverse to Epochs

In [7]:
epochs.set_eeg_reference(projection=True) # needed for inverse modelling, ignore error about filename due to selfmade info

snr = 3.0
lambda2 = 1.0 / snr**2
stcs = mne.minimum_norm.apply_inverse_epochs(
    epochs,
    inverse_operator,
    lambda2=lambda2,
    method='MNE',
    pick_ori='vector'
)

EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    The projection vectors do not apply to these channels.
    Created the whitener using a noise covariance matrix with rank 64 (0 small eigenvalues omitted)
Picked 64 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 5
Processing epoch : 2 / 5
Processing epoch : 3 / 5
Processing epoch : 4 / 5
Processing epoch : 5 / 5
[done]


In [8]:
for stc in stcs:
    print(stc)

<VectorSourceEstimate | 20484 vertices, subject : fsaverage, tmin : -1000.0 (ms), tmax : 4996.09375 (ms), tstep : 3.90625 (ms), data shape : (20484, 3, 1536), ~720.3 MiB>
<VectorSourceEstimate | 20484 vertices, subject : fsaverage, tmin : -1000.0 (ms), tmax : 4996.09375 (ms), tstep : 3.90625 (ms), data shape : (20484, 3, 1536), ~720.3 MiB>
<VectorSourceEstimate | 20484 vertices, subject : fsaverage, tmin : -1000.0 (ms), tmax : 4996.09375 (ms), tstep : 3.90625 (ms), data shape : (20484, 3, 1536), ~720.3 MiB>
<VectorSourceEstimate | 20484 vertices, subject : fsaverage, tmin : -1000.0 (ms), tmax : 4996.09375 (ms), tstep : 3.90625 (ms), data shape : (20484, 3, 1536), ~720.3 MiB>
<VectorSourceEstimate | 20484 vertices, subject : fsaverage, tmin : -1000.0 (ms), tmax : 4996.09375 (ms), tstep : 3.90625 (ms), data shape : (20484, 3, 1536), ~720.3 MiB>


In [9]:
# 6. Read Labels and Extract ROI Time Series
if "fsaverage" in os.path.basename(subjects_dir):
    subjects_dir = os.path.dirname(subjects_dir)  # Move one directory up
labels = mne.read_labels_from_annot(
    subject='fsaverage',
    parc='aparc',
    subjects_dir=subjects_dir
)

src = mne.read_source_spaces(os.path.join(subjects_dir, 'bem', 'fsaverage-ico-5-src.fif'))  # same src as forward
stcs_label_ts = mne.extract_label_time_course(
    stcs, labels, src, mode='mean_flip'
)
# stcs_label_ts is a list of arrays, each shaped (n_labels, n_times)
# One per epoch.

print(f"Extracted label time courses for {len(stcs_label_ts)} epochs.")
print(f"Shape of the first epoch's ROI matrix: {stcs_label_ts[0].shape}")

Reading labels from parcellation...
   read 35 labels from /Users/s204684/mne_data/MNE-fsaverage-data/fsaverage/label/lh.aparc.annot
   read 34 labels from /Users/s204684/mne_data/MNE-fsaverage-data/fsaverage/label/rh.aparc.annot


FileNotFoundError: File does not exist: "/Users/s204684/mne_data/MNE-fsaverage-data/bem/fsaverage-ico-5-src.fif"

## 4. Concatenate Epochs into 1 raw

In [ ]:
# 1) Get data shape
n_epochs, n_channels, n_times = epochs.get_data().shape
print(f"Epochs shape: {n_epochs} epochs, {n_channels} channels, {n_times} time points each")

# 2) Grab only EEG channel data (ignore EOG, etc. if needed)
#    If your epochs contain EOG/ECG references, you can do e.g.:
epochs_eeg = epochs.copy().pick_types(eeg=True)

# 3) Convert [n_epochs, n_channels, n_times] -> [n_channels, n_epochs * n_times]
data_3d = epochs_eeg.get_data()               # shape (n_epochs, n_eeg_ch, n_times)
data_2d = data_3d.transpose(1, 0, 2).reshape(n_channels, -1)

# 4) Create RawArray with the same Info structure (minus non-EEG channels)
info_eeg = epochs_eeg.info  # has correct channel names, types, etc.
raw = mne.io.RawArray(data_2d, info_eeg)
raw._filenames = [""]  # to avoid potential filename-related warnings
raw.set_eeg_reference(projection=True) # needed for inverse modelling, ignore error about filename due to selfmade info
print(raw)

In [ ]:
# 1) Diagonal ad-hoc noise covariance
noise_cov = mne.make_ad_hoc_cov(raw.info, None)

# 2) Build inverse operator
#    (You already have fwd from your script)
inverse_operator = mne.minimum_norm.make_inverse_operator(
    info=raw.info,        # important to use the same sensor info as `raw`
    forward=fwd,          # your precomputed forward
    noise_cov=noise_cov,
    loose=1.0,            # free orientation
    depth=0.8
)


In [ ]:
if "fsaverage" in os.path.basename(subjects_dir):
    subjects_dir = os.path.dirname(subjects_dir)  # Move one directory up

labels = mne.read_labels_from_annot("fsaverage", parc="aparc",
                                    subjects_dir=subjects_dir)
# Optionally remove 'unknown' or medial wall labels if they exist
labels = [lbl for lbl in labels if lbl.name.lower() != 'unknown-lh']
n_labels = len(labels)
print(f"Number of labels (ROIs): {n_labels}")

# 2) Define the regularization parameter for MNE inverse
snr = 3.0
lambda2 = 1.0 / snr**2

# 3) Preallocate an array to store label time series
#    We'll have shape = [n_labels, total_time_points]
#    total_time_points = n_epochs * n_times
label_ts = np.zeros((n_labels, n_epochs * n_times))

for li, label in enumerate(labels):
    # Apply inverse only for this label’s vertices
    stc = mne.minimum_norm.apply_inverse_raw(
        raw,                 
        inverse_operator,    
        lambda2=lambda2,     
        method='MNE',        
        pick_ori='vector',   # unconstrained 3D orientation
        label=label,         # <--- key: only invert for these vertices
        verbose=False
    )

    # Optionally reduce 3 orientations with PCA
    stc_pca, pca_dir = stc.project(directions='pca', src=inverse_operator['src'])

    # Extract label time course (mean_flip helps handle polarity flips)
    roi_data = mne.extract_label_time_course(
        stc_pca, [label], inverse_operator['src'],
        mode='mean_flip', return_generator=False
    )
    # roi_data has shape (1, total_time_points), since we used a single label
    # squeeze or index [0] to get (total_time_points, )
    label_ts[li, :] = roi_data[0, :]

    # Free memory
    del stc, stc_pca

    if (li+1) % 5 == 0:
        print(f"Processed {li+1} / {n_labels} labels")

print("All labels processed. Shape of label_ts:", label_ts.shape)


In [ ]:
# Suppose label_ts is [n_labels, n_epochs*n_times]
# We want => [n_epochs, n_labels, n_times]
label_ts_reshaped = label_ts.reshape(
    n_labels, n_epochs, n_times
).transpose(1, 0, 2)

print("New shape: ", label_ts_reshaped.shape)  # (n_epochs, n_labels, n_times)


In [ ]:
# make an epoch object
info = mne.create_info(
    ch_names=[label.name for label in labels],
    sfreq=epochs.info['sfreq'],
    ch_types='eeg'
)
label_epochs = mne.EpochsArray(
    data=label_ts_reshaped,
    info=info,
    tmin=epochs.times[0],
    verbose=False
)


In [ ]:
%matplotlib qt
label_epochs.plot(n_channels=68, n_epochs=10)
